In [1]:
import xlearn as xl

In [2]:
xl.hello()

In [15]:
import os
from pathlib import Path

import pandas as pd

In [16]:
DATA_PATH = Path('..') / '..' / 'data' / 'criteo'

TRAIN_PATH = str(DATA_PATH / 'train.csv')

In [18]:
train = pd.read_csv(TRAIN_PATH)

In [19]:
num_columns = ['_c{}'.format(i) for i in range(1, 14)]
cat_columns = ['_c{}'.format(i) for i in range(14, 40)]
len(num_columns), len(cat_columns)

(13, 26)

In [20]:
import category_encoders as ce

In [21]:
encoder = ce.CatBoostEncoder(cols=cat_columns)

In [40]:
X, y = train.drop(columns=['_c0', 'id']), train['_c0']

In [23]:
%%time
X_encoded = encoder.fit_transform(X, y)

CPU times: user 9min 35s, sys: 46.5 s, total: 10min 21s
Wall time: 5min 10s


In [32]:
X_encoded[num_columns] = X_encoded[num_columns].fillna(0)

In [33]:
from sklearn.preprocessing import StandardScaler

In [34]:
scaler = StandardScaler()

In [35]:
X_encoded[num_columns] = scaler.fit_transform(X_encoded[num_columns])

In [45]:
fm_model = xl.FMModel(task='binary', init=0.1, 
                      epoch=10, k=4, lr=0.1, 
                      reg_lambda=0.01, opt='sgd', 
                      metric='acc')

In [46]:
fm_model.fit(X_encoded.to_numpy(), y)

In [48]:
TEST_PATH = os.path.join(DATA_PATH, 'test.csv')

In [49]:
kaggle = pd.read_csv(TEST_PATH)

In [50]:
kaggle, id_kaggle = kaggle.drop(columns='id'), kaggle['id']

In [51]:
%%time
kaggle_encoded = encoder.transform(kaggle)

CPU times: user 1min 2s, sys: 6.11 s, total: 1min 8s
Wall time: 16.3 s


In [54]:
kaggle_encoded[num_columns] = kaggle_encoded[num_columns].fillna(0)

In [55]:
kaggle_encoded[num_columns] = scaler.transform(kaggle_encoded[num_columns])

In [56]:
preds = fm_model.predict(kaggle_encoded.to_numpy())

In [57]:
kaggle['proba'] = preds

In [58]:
kaggle['id'] = id_kaggle

In [59]:
SUBMISSION_PATH = 'FMM_full_dataset.csv'

In [60]:
kaggle[["id", "proba"]].to_csv(SUBMISSION_PATH, index=False)

In [66]:
! kaggle competitions submit mlbd-20-ctr-prediction-1 -f $SUBMISSION_PATH -m "full train, FMM"

100%|######################################| 19.2M/19.2M [00:03<00:00, 6.39MB/s]
Successfully submitted to CTR prediction

In [67]:
! kaggle competitions submissions mlbd-20-ctr-prediction-1 -v | grep FMM

FMM_full_dataset.csv,2020-04-01 00:06:25,"full train, FMM",complete,0.65786,0.65948
